# ANALISE DE CRM

A ideia desse projeto e realizar uma analise de dados baseada em um dataset de um CRM exemplo. O dataset pode ser baixado [aqui](https://www.kaggle.com/datasets/carrie1/ecommerce-data?resource=download)

## IMPORT DE PACOTES
Nessa secao, serao importados os pacotes necessarios para realizar uma analise do dataset. Sao eles:
- pandas
- plotly

In [2]:
import pandas
import plotly.express as px

## IMPORT DATASET

In [3]:
df_crm = pandas.read_csv('../assets/data.csv', encoding= 'unicode_escape')
df_crm.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


## ORGANIZACAO E LIMPEZA DOS DADOS
Os datasets podem conter dados fora do formato que necessario para a analise e pode conter nulos em alguma coluna devido algum erro na insercao dos dados na ferramenta. Alem disso pode existir outliers que algumas vezes nao fazem sentido analisar por terem sido muito pontuais ou gerados por algum erro sistemico.

In [4]:
df_crm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [5]:
df_crm['InvoiceDate'] = df_crm['InvoiceDate'].astype('datetime64[ns]')
df_crm['CustomerID'] = df_crm['CustomerID'].astype('str')
df_crm.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   541909 non-null  object        
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(5)
memory usage: 33.1+ MB


## ANALISE DE DADOS
Toda analise de dados precisa ter um objetivo em mente.

### Encontrar a Serie Historica de Vendas

In [12]:
df_historical_sales = df_crm[['InvoiceDate', 'Quantity', 'UnitPrice']]
df_historical_sales['TotalValue'] = df_historical_sales['Quantity'] * df_historical_sales['UnitPrice']

df_historical_sales.head()

C:\Users\Guilh\AppData\Local\Temp\ipykernel_13736\2335235470.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_historical_sales['TotalValue'] = df_historical_sales['Quantity'] * df_historical_sales['UnitPrice']


,InvoiceDate,Quantity,UnitPrice,TotalValue
0,2010-12-01 08:26:00,6,2.55,15.30
1,2010-12-01 08:26:00,6,3.39,20.34
2,2010-12-01 08:26:00,8,2.75,22.00
3,2010-12-01 08:26:00,6,3.39,20.34
4,2010-12-01 08:26:00,6,3.39,20.34


In [25]:
def format_number(x):
    if(x < 10):
        return '0'+x
    else:
        return x

df_historical_sales_month = df_historical_sales
df_historical_sales_month['YearMonth'] = pandas.to_datetime(df_historical_sales_month['InvoiceDate']).apply(lambda x: '{year}-{month}'.format(year=x.year, month=format_number(x.month)))
df_historical_sales_month_result = df_historical_sales_month.groupby('YearMonth')['TotalValue'].sum()
df_plot = df_historical_sales_month_result.to_frame().reset_index().sort_values(by=['YearMonth'])

df_plot


C:\Users\Guilh\AppData\Local\Temp\ipykernel_13736\2568633414.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,YearMonth,TotalValue
0,2010-12,748957.020
1,2011-1,560000.260
2,2011-10,1070704.670
3,2011-11,1461756.250
4,2011-12,433668.010
5,2011-2,498062.650
6,2011-3,683267.080
7,2011-4,493207.121
8,2011-5,723333.510
9,2011-6,691123.120


In [23]:
fig = px.line(df_plot, x="YearMonth", y="TotalValue", title='Sales per Month')
fig.show()
